https://doc.sagemath.org/html/en/reference/rings_numerical/sage/rings/real_mpfi.html

https://doc.sagemath.org/html/en/reference/spkg/mpfi.html#spkg-mpfi

https://doc.sagemath.org/html/en/reference/spkg/mpfr.html#spkg-mpfr

In [1]:
RealIntervalField(53) is RIF

True

In [2]:
a=RIF(1,2)
print(a==a)

False


In [3]:
R = RealIntervalField(10)

print(R(pi)==R(pi))
print(R(pi).str(error_digits=2))

RIF(54321/256).str(error_digits=1)

False
3.1426?20


'212.19140625000000?0'

In [4]:
R = RealIntervalField(3)
a = R(1.25)
print(a.str(style='brackets'))
print(a==5/4)

R = RealIntervalField(1)
a = R(1.25)
print(a.str(style='brackets'))
print(a==5/4)

[1.2 .. 1.3]
True
[1.0 .. 2.0]
False


Comparison operations (==, !=, <, <=, >, >=) return True if every value in the first interval has the given relation to every value in the second interval.

In [5]:
print(0 == RIF(0,1))
print(0 != RIF(0,1))
print(RIF(0,oo) == oo)
print(RIF(-oo,oo) == oo)

False
False
True
True


In [6]:
from sage.rings import real_mpfi

real_mpfi.printing_style = 'brackets'
print(RIF((1/3,2/3)))

real_mpfi.printing_style = 'question'
print(RIF((1/3,2/3)))
print(RIF((1/3,2/3)).str(style='brackets'))


[0.33333333333333331 .. 0.66666666666666675]
1.?
[0.33333333333333331 .. 0.66666666666666675]


In [7]:
print((RealIntervalField(53)(1.5,2.5)+RealIntervalField(21)(1.5,2.5)))
print((RealIntervalField(53)(1.5,2.5)+RealIntervalField(21)(1.5,2.5)).str(style='brackets'))
print((RealIntervalField(53)(1.5,2.5)+RealIntervalField(21)(1.5,2.5)).precision())
print((RealIntervalField(53)(1.5,2.5)+RealIntervalField(21)(1.5,2.5)).lower())
print((RealIntervalField(53)(1.5,2.5)+RealIntervalField(21)(1.5,2.5)).upper())

4.?
[3.0000000 .. 5.0000000]
21
3.00000
5.00000


In [8]:
print(R(pi))
print(R(pi)/3)

3.?
1.?


In [ ]:
prec = 300
R = RealBallField(prec)
var('x')
assume(x, 'real')
var('y')
assume(y, 'real')

f = x^2
g=f(x=y)

print(f+g)

x^2 + y^2


In [1]:
R = RealBallField(100)

x1=R(pi)
x2=R(pi^2)
n=10
print( (x1-x2)/n)
print((x1-x2)/R(n))

[-0.67280117474995653803718476166 +/- 5.42e-30]
[-0.67280117474995653803718476166 +/- 5.42e-30]


In [5]:
x=pi 
print(x)
x=R(x)
print(x)

pi
[3.14159265358979323846264338328 +/- 2.25e-30]
